# Linear Modeling with `statsmodels`
---

As we've seen, `scikit-learn` is an essential package but geared towards machine learning rather than statistics.  The `statsmodels` package brings more traditional statistics functionality to Python.  The two can be used together in various ways, so knowing both is handy for complex tasks.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.datasets import *
from sklearn.preprocessing import *

%matplotlib inline

## Linear Regression
---

Let's re-visit the Boston housing dataset, loading it via `load_boston()`, creating a data frame of features and a series of labels, then combining them together with `concat()`:

In [ ]:
boston = load_boston()

features_df = pd.DataFrame(boston['data'], columns = boston['feature_names'])
labels = pd.Series(boston['target'], name = 'MEDV')

training_df = pd.concat([features_df, labels], axis = 1)
training_df.head()

The `statsmodels` package has a different API than `scikit-learn` and so takes some getting used to.  Let's create an OLS (ordinary linear regression) class from a formula that specifies the dependent variable on the left of a tilde (`~`) with the independent variables on the right, separated by plus signs.  We also tell it which data frame to reference those names in via the `data` parameter.

After creating a model, we call `fit()` but don't need to pass any parameters here.  Finally, we call `summary()` on the fit object to get R-like output:

In [ ]:
model = smf.ols(formula = 'MEDV ~ RM + RAD', data = training_df)
fit = model.fit()
fit.summary()

Seems comparable to before -- let's see how it performs using all predictors:

In [ ]:
model = smf.ols(formula = 'MEDV ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT', data = training_df)
fit = model.fit()
fit.summary()

This also seems comparable.  In R, there is a nice shortcut enabled by formulas where listing a period to the right of a tilde will use all columns as predictors that aren't on the left.  Due to limitations in `statsmodels`, we can't do that, so I provided a function here that will generate such a formula:

In [ ]:
def get_formula(variables, dependent_variable_name):
    independent_variables = set(variables) - set([dependent_variable_name])
    return dependent_variable_name + ' ~ ' + ' + '.join(independent_variables)

In [ ]:
formula = get_formula(training_df.columns, 'MEDV')
formula

In [ ]:
model = smf.ols(formula = formula, data = training_df)
fit = model.fit()
fit.summary()

## Residual Diagnostics
---

There are various diagnostic plots for checking the assumptions of the model. One of these, `plot_regress_exog()`, plots a 2x2 grid of figures showing the true and predicted labels vs the independent variable, residuals vs the independent variable, a partial regression plot, and a component-component plus residual (CCPR) plot.

See the official [`statsmodels` notebooks](http://www.statsmodels.org/stable/examples/notebooks/generated/regression_plots.html) for more examples.

In [ ]:
sm.graphics.plot_regress_exog(fit, 'RM', fig = plt.figure(figsize = (12, 12)));

## Logistic Regression
---

We can perform classification, where the dependent variable is binary rather than continuous, using *logistic regression*.  A common dataset for demonstrating logistic regression is the breast cancer dataset describing malignant and benign tumors:

In [ ]:
bc = load_breast_cancer()

safe_columns = [_.replace(' ', '_') for _ in bc['feature_names']]
features_df = pd.DataFrame(bc['data'], columns = safe_columns)
labels = pd.Series(bc['target'], name = 'malignant')

features_df = features_df.loc[:, ~features_df.columns.str.startswith('worst')]

training_df = pd.concat([features_df, labels], axis = 1)
training_df.info()

For simplicity we've dropped a set of features beginning with the word `worst`.

Our code is the same excelt we call the `glm()` function, and set the `family` parameter in order to apply the logit transformation necessary for binary classification:

In [ ]:
model = smf.glm(
    formula = get_formula(training_df.columns, 'malignant'),
    data = training_df,
    family = sm.families.Binomial()
)
fit = model.fit()
fit.summary()

## Logistic Regression with `scikit-learn`
---

It's quite simple to apply logistic regression using `scikit-learn` instead.  We simply use the `LogisticRegression()` class rather than `LinearRegression()`, and we use different performance metrics such as accuracy, precision, or recall to evaluate the model.

Avoid using accuracy whenever possible, due to issues discussed in depth in advanced classes.

In [ ]:
from sklearn.linear_model import *
from sklearn.metrics import *

In [ ]:
model = LogisticRegression()

model.fit(features_df, labels)
predictions = model.predict(features_df)

print(accuracy_score(labels, predictions))
print(precision_score(labels, predictions))
print(recall_score(labels, predictions))

In [ ]:
coefs = pd.Series(model.coef_[0], index = features_df.columns).sort_values()
print(coefs.head())
print()
print(coefs.tail())

Notice the coefficients don't match those from R: `scikit-learn` uses different methods for the optimization problem that learns the coefficients, so the performance will likely be similar but the coefficients may not be.

## Calling R from Python
---

Finally, we can compare our output directly to R from within Python.  In fact, using the `rpy2` package, we can call any R function and convert back and forth between Python and R data types.

We first load the `broom` package in R to convert the output of `glm` into a data frame so the results returned to Python are easily readable.  We then call the `pandas2ri.activate()` function which automatically translates Pandas data frames to R data frames and back.  Finally, we fit a logistic regression model (`glm()` with `family = 'binomial'`) and print the summary statistics.

In [ ]:
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr

importr('broom')

pandas2ri.activate()

fit = r.glm(
    'malignant ~ .',
    data = training_df,
    family = 'binomial'
)
print(r.tidy(fit))

Unlike `scikit-learn`, these coefficients should match our `statsmodels` output above.